https://gist.github.com/sloria/6407257

In [2]:
import math
from textblob import TextBlob as tb

tf(word, blob) computes "term frequency" which is the number of times a word appears in a document blob, normalized by dividing by the total number of words in blob. We use TextBlob for breaking up the text into words and getting the word counts.

In [3]:
def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

n_containing(word, bloblist) returns the number of documents containing word. A generator expression is passed to the sum() function.

Le n_containing correspond au bm.df du bm25

In [4]:
def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob)

idf(word, bloblist) computes "inverse document frequency" which measures how common a word is among all documents in bloblist. The more common a word is, the lower its idf. We take the ratio of the total number of documents to the number of documents containing word, then take the log of that. Add 1 to the divisor to prevent division by zero.

In [5]:
def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

tfidf(word, blob, bloblist) computes the TF-IDF score. It's the product of tf and idf.

In [6]:
def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

## Un exemple concret

In [7]:
expos = ["L'ex-ami de sa voisine (Mme LADRIERE, rez-de-chaussée) fait du scandale dans le bâtiment. S'agit d'un dénommé LOSET.",
 "Un voisin s'engeule avec les client, filme tout le monde et hurle.",
 "Un requérant sous l'influence de l'alcool crée du trouble dans le centre. Se montre menaçant verbalement",
 "les esprits s'échauffent devant le magasin, entre 2 groupes de jeunes",
 'Code entrée 6184, ne veut plus de son ami chez elle. Ce dernier ivre, fait du scandale dans le corridor.',
 'un homme crée du scandale à la réception de la caisse de Chômage',
 "Des jeunes font du scandale au bas de l'immeuble, à côté d'un bar.",
 '1 client alcoolisé sème la zizanie dans le secteur',
 'Signale des individus avinés qui font du grabuge sur le quai 5.',
 "Par une trentaine de personnes, vers l'air de la station."]

In [8]:
bloblist = [tb(expo) for expo in expos]

In [9]:
len(bloblist)

10

In [28]:
bloblist[0]

TextBlob("L'ex-ami de sa voisine (Mme LADRIERE, rez-de-chaussée) fait du scandale dans le bâtiment. S'agit d'un dénommé LOSET.")

In [29]:
print(type(bloblist[0]))

<class 'textblob.blob.TextBlob'>


In [30]:
print(bloblist[0])

L'ex-ami de sa voisine (Mme LADRIERE, rez-de-chaussée) fait du scandale dans le bâtiment. S'agit d'un dénommé LOSET.


For each document, we store the TF-IDF scores in a dictionary scores mapping word => score using a dict comprehension. We then sort the words by their scores and output the top 3 words.

In [31]:
bloblist[0].words

WordList(["L'ex-ami", 'de', 'sa', 'voisine', 'Mme', 'LADRIERE', 'rez-de-chaussée', 'fait', 'du', 'scandale', 'dans', 'le', 'bâtiment', "S'agit", "d'un", 'dénommé', 'LOSET'])

In [33]:
l = list(bloblist[0].words)
l

["L'ex-ami",
 'de',
 'sa',
 'voisine',
 'Mme',
 'LADRIERE',
 'rez-de-chaussée',
 'fait',
 'du',
 'scandale',
 'dans',
 'le',
 'bâtiment',
 "S'agit",
 "d'un",
 'dénommé',
 'LOSET']

In [26]:
for i, blob in enumerate(bloblist):
    print("Top words in expo {}".format(i + 1))
    scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:3]:
        print("Word: {}, TF-IDF: {}".format(word, round(score, 5)))
    print("*******************************")

Top words in expo 1
Word: L'ex-ami, TF-IDF: 0.09467
Word: sa, TF-IDF: 0.09467
Word: voisine, TF-IDF: 0.09467
*******************************
Top words in expo 2
Word: s'engeule, TF-IDF: 0.13412
Word: avec, TF-IDF: 0.13412
Word: filme, TF-IDF: 0.13412
*******************************
Top words in expo 3
Word: requérant, TF-IDF: 0.10059
Word: sous, TF-IDF: 0.10059
Word: l'influence, TF-IDF: 0.10059
*******************************
Top words in expo 4
Word: esprits, TF-IDF: 0.14631
Word: s'échauffent, TF-IDF: 0.14631
Word: devant, TF-IDF: 0.14631
*******************************
Top words in expo 5
Word: Code, TF-IDF: 0.08047
Word: entrée, TF-IDF: 0.08047
Word: 6184, TF-IDF: 0.08047
*******************************
Top words in expo 6
Word: la, TF-IDF: 0.14097
Word: homme, TF-IDF: 0.1238
Word: réception, TF-IDF: 0.1238
*******************************
Top words in expo 7
Word: Des, TF-IDF: 0.1238
Word: bas, TF-IDF: 0.1238
Word: l'immeuble, TF-IDF: 0.1238
*******************************
Top wor